In [1]:
from datasets import load_dataset, DatasetDict
from os import environ
environ["HF_DATASETS_CACHE"] = "/cache/peterr/HFCache"
environ["CUDA_VISIBLE_DEVICES"] = "1"

ds = load_dataset(
    "5roop/ParlaSpeech-HR", split="test", streaming="True"
)
# We will only take the first 10 items here
first_ten = ds.take(10)


# This part converts IterableDataset to ordinary dataset:
from functools import partial
from datasets import Dataset
def gen_from_iterable_dataset(iterable_ds):
    yield from iterable_ds
ds = Dataset.from_generator(partial(gen_from_iterable_dataset, first_ten), features=first_ten.features)

# The rest is just how I transcribe
# Note that we need to transform the dataset to a KeyDataset with key being 'audio' column
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from pathlib import Path

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v3"
try:
    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id,
        torch_dtype=torch_dtype,
        low_cpu_mem_usage=True,
        use_safetensors=True,
    )
except:
    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id,
        torch_dtype=torch_dtype,
        low_cpu_mem_usage=True,
        use_safetensors=False,
    )
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)


from transformers.pipelines.pt_utils import KeyDataset
result = pipe(
    KeyDataset(ds, "audio"),
    generate_kwargs={"language": "croatian"},
)
transcripts = [i.get("text") for i in result]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
transcripts

[' ali definitivno predstavljaju dio rješenja i Republika Hrvatska je u jednoj prednosti u odnosu na neke druge zemlje u Srednjoj Evropi i na Zapadu i na Istoku zato što ima 25% društvenog bruto proizvoda u mirovinskim fondovima što nas već sad obilježava kao srednje razvijenu zemlju što',
 ' mu je iznosila oko 24,5 milijarde kuna i u fondu je bilo 532 tisuće članova koncem godine. Prinos koji je ostvaren u 2016. godini je 7,31%, a od početka rada,',
 ' Glede same državne komisije treba istaknuti da temelju činkovitosti rada državne komisije predstavlja stalnost ljudskih resursa na stručnim poslovima visoke složenosti, koja omogućava razvoj stručnosti i znanja, a što dovodi do skraćivanja rokova odlučivanja, veće efikasnosti i kvalitete odluka. Državna komisija kao hvazi sudbeno tijelo i autoritetu',
 ' Ključno je obrazovanje i mislim da je važno i ovdje istaknuti da je Ministarstvo obrazovanja za školsku godinu 2017-2018 osiguralo 2317 pomoćnika u nastavi koji pružaju uslugu za 2670 u

In [3]:
ds["transcript"] # This is the original non-normalised transcript

['ali definitivno predstavljaju dio rješenja i RH, je u jednoj prednosti u odnosu na neke druge zemlje u srednjoj Europi i na zapadu i na istoku zato što ima 25% društvenog bruto proizvoda u mirovinskim fondovima što nas već sad obilježava kao srednje razvijenu zemlju što',
 'mu je iznosila oko 24 i pol milijarde kuna i u fondu je bilo 532 tisuće članova koncem godine. Prinos koji je ostvaren u 2016. godini je 7,_31%,a od početka rada,',
 'Glede same državne komisije treba istaknuti da temelj učinkovitosti rada državne komisije predstavlja stalnost ljudskih resursa na stručnim poslovima visoke složenosti koja omogućava razvoj stručnosti i znanja, a što dovodi do skraćivanja rokova odlučivanja, veće efikasnosti i kvalitete odluka. Državna komisija kao kvazi sudbeno tijelo i autoritet u',
 'ključno jest obrazovanje i mislim da je važno i ovdje istaknuti da je Ministarstvo obrazovanja za školsku godinu 2017., 2018. osiguralo 2317 pomoćnika u nastavi koji pružaju uslugu za 2670 učenika s t

In [21]:
from datasets import load_dataset, DatasetDict
from os import environ
environ["HF_DATASETS_CACHE"] = "/cache/peterr/HFCache"
environ["CUDA_VISIBLE_DEVICES"] = "1"

from functools import partial
from datasets import Dataset
def gen_from_iterable_dataset(iterable_ds):
    yield from iterable_ds
    
ds = load_dataset(
    "5roop/ParlaSpeech-HR", split="test", streaming="True"
)
CHUNKSIZE = 100
STARTINDEX = 0
texts = []
asr_results = []
while True:
    chunk = ds.skip(STARTINDEX).take(CHUNKSIZE)
    if list(chunk) == []:
        break
    chunk = Dataset.from_generator(partial(gen_from_iterable_dataset, chunk), features=chunk.features)
    texts.extend(chunk["transcript"])
    asr_results_for_this_chunk = ["Generate", "This", "List", "with", "Your", "Model", ...]
    asr_results.extend(asr_results_for_this_chunk)
    
    # In the next step we will have to skip more instances to move our chunk forward:
    STARTINDEX += CHUNKSIZE
    break